In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
import numpy as np
from GradCAM import *
from define_model import *
from load_data import *
from utilities import *
from Parse_TFrecords import *
 
print(tf.__version__)

In [ ]:
with tf.device("cpu:0"): 
    checkpoint_filepath = 'checkpoints/AUC/checkpoint_BCE_Dnet121'
    archi = 'Dnet121'
    model = define_model(archi)
    model.load_weights(checkpoint_filepath)

# GradCAM

In [ ]:
with tf.device("cpu:0"): 
    last_conv_layer_name = 'relu'
    classifier_layer_names = ['max_pool', 'dense']

    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(classifier_input, x)

In [ ]:
record_file_test = 'copd_test_new.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=256, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train))

In [ ]:
heat_map = []
preds = []
img = []
with tf.device("cpu:0"): 
    for i, l in test_dataset:
        if (l == 1):
            img_array = np.reshape(i, (1, 256, 256, 3))
            pred = model.predict(img_array)
            if(pred >= 0.9):
                img.append(img_array)
                heat_map.append(show_heatmap(img_array, last_conv_layer_model, classifier_model))
                preds.append(pred)
                
    preds = np.array(preds).reshape(len(preds,))
    order = np.argsort(np.array(preds))

In [ ]:
for i in order[-20:]:
    plt.imshow(np.reshape(heat_map[i], (256, 256, 3)))
    plt.show()

In [ ]:
mean_heatmap = np.mean(heat_map, axis=0)
plt.imshow(np.reshape(mean_heatmap, (256, 256, 3)))
plt.show()

# Saliency map

In [ ]:
with tf.device("cpu:0"):
    def get_saliency_map(model, image):
        with tf.GradientTape() as tape:
            tape.watch(image)
            loss = model(image)
                        
        # Get the gradients of the loss w.r.t to the input image.
        gradient = tape.gradient(loss, image)
                
        dgrad_abs = tf.math.abs(gradient)
#         print(dgrad_abs.shape)
        dgrad_max_ = np.max(dgrad_abs, axis=-1)[0]
#         print(dgrad_max_.shape)

        # normaliz between 0 and 1
        arr_min, arr_max  = np.min(dgrad_max_), np.max(dgrad_max_)
        smap = (dgrad_max_ - arr_min) / (arr_max - arr_min + 1e-18)

        return smap

In [ ]:
with tf.device("cpu:0"):
    linear_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-1].output)
    for i in order[-20:]:
        print(preds[i])
        smap = get_saliency_map(linear_model, tf.Variable(img[i], dtype=float))

        plt.subplot(1, 2, 1)
        plt.imshow(img[i][0])

        plt.subplot(1, 2, 2)
        plt.imshow(smap, cmap='Reds')

        plt.show()

In [ ]:
with tf.device("cpu:0"):
    smaps = []
    for i in range(len(img)):
        smaps.append(get_saliency_map(linear_model, tf.Variable(img[i], dtype=float)))

In [ ]:
mean_img = np.mean(img, axis=0)
plt.subplot(1, 2, 1)
plt.imshow(mean_img[0], cmap='gray')

mean_smap = np.mean(smaps, axis=0)
plt.subplot(1, 2, 2)
plt.imshow(mean_smap, cmap='Reds')

plt.show()